In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from lightning_utils import *
from utils import *
from models import (TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, 
                    ComplexTorchMLP, ComplexSymPyModule, complex_mse)
from models import RobustPCANN
from pytorch_robust_pca import *
from preprocess import *

# Model selection
# from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

from madgrad import MADGRAD
import lookahead

# BayesianOptimization
from bayes_opt import BayesianOptimization
from skopt import Optimizer

from tqdm import trange

Running Python 3.9.7
You can use npar for np.array


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Adding noise
noisy_lables = True
noisy_xt = True
noise_intensity = 0.0
if noisy_lables: noise_intensity = 0.01/np.sqrt(2) 

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = to_column_vector(Exact_u.T)
v_star = to_column_vector(Exact_v.T)

N = 500
idx = np.random.choice(X_star.shape[0], N, replace=False)
# idx = np.arange(N) # Just have an easy dataset for experimenting

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

# if noisy_xt:
#     print("Noisy (x, t)")
#     X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
# else: print("Clean (x, t)")

# X_train = to_tensor(X_star[idx, :], True).to(device)
# u_train = to_tensor(u_star[idx, :], False).to(device)
# v_train = to_tensor(v_star[idx, :], False).to(device)

feature_names = ['hf', '|hf|', 'h_xx']

### Loading (clean) data code here ###
print("Loading pre-calculated (clean) data for reproducibility")
X_train = np.load("./tmp_files/X_train_500+500samples.npy")

if noise_intensity > 0.0 and noisy_xt:
    print("Noisy (x, t)")
    X_train = perturb(X_train, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

X_train = to_tensor(X_train, True)[:N, :]

uv_train = np.load("./tmp_files/uv_train_500samples.npy")
u_train = uv_train[:, 0:1]; v_train = uv_train[:, 1:2]

You're running on cpu
Loading pre-calculated (clean) data for reproducibility
Noisy (x, t)


In [3]:
if noise_intensity > 0.0:
    noise_u = perturb(u_train, intensity=noise_intensity, noise_type="normal", overwrite=False)
    u_train = u_train + noise_u
    noise_v = perturb(v_train, intensity=noise_intensity, noise_type="normal", overwrite=False)
    v_train = v_train + noise_v
    print("Perturbed u_train and v_train with intensity =", float(noise_intensity))
u_train = u_train[:N, :]; v_train = v_train[:N, :]

u_train, v_train = to_tensor(u_train, False), to_tensor(v_train, False)
h_train = torch.complex(u_train, v_train)

Perturbed u_train and v_train with intensity = 0.0070710678118654745


In [4]:
cn1 = -0.020763+1.029535*1j
cn2 = -0.016447+0.509700*1j
cns = [cn1, cn2]

In [5]:
# Type the equation got from the symbolic regression step
# No need to save the eq save a pickle file before
program1 = "X0*X1"
pde_expr1, variables1,  = build_exp(program1); print(pde_expr1, variables1)

program2 = "X2"
pde_expr2, variables2,  = build_exp(program2); print(pde_expr2, variables2)

mod = ComplexSymPyModule(expressions=[pde_expr1, pde_expr2], complex_coeffs=cns); mod.train()

X0*X1 {X0, X1}
X2 {X2}


ComplexSymPyModule(
  (sympymodule): SymPyModule(expressions=(X0*X1, X2))
)

In [6]:
class ComplexPINN(nn.Module):
    def __init__(self, model, loss_fn, index2features, scale=False, lb=None, ub=None):
        super(ComplexPINN, self).__init__()        
        self.model = model
        self.callable_loss_fn = loss_fn
        self.index2features = index2features; self.feature2index = {}
        for idx, fn in enumerate(self.index2features): self.feature2index[fn] = str(idx)
        self.scale = scale; self.lb, self.ub = lb, ub
        if self.scale and (self.lb is None or self.ub is None):
            print("Please provide thw lower and upper bounds of your PDE.")
            print("Otherwise, there will be error(s)")
        self.diff_flag = diff_flag(self.index2features)
        
    def forward(self, H):
        if self.scale: H = self.neural_net_scale(H)
        return self.model(H)
    
    def loss(self, HL, y_input, update_network_params=True, update_pde_params=True):
        total_loss = []
        
        # Forwarding
        grads_dict, u_t = self.grads_dict(HL[:, 0:1], HL[:, 1:2])
        
        # MSE Loss
        if update_network_params:
            total_loss.append(complex_mse(grads_dict['X'+self.feature2index['hf']], y_input))
        # PDE Loss
        if update_pde_params:
            total_loss.append(complex_mse(self.callable_loss_fn(grads_dict), u_t))
            
        return total_loss
    
    def grads_dict(self, x, t):
        uf = self.forward(cat(x, t))
        u_t = complex_diff(uf, t)
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = {}
        for t in self.diff_flag[0]:
            if t=='hf': 
                derivatives['X'+self.feature2index[t]] = cplx2tensor(uf)
                derivatives['X1'] = (uf.real**2+uf.imag**2)+0.0j
            elif t=='x': derivatives['X'+self.feature2index[t]] = x
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = complex_diff(out, x)
                elif c=='t': out = complex_diff(out, t)
            derivatives['X'+self.feature2index['h_'+t[::-1]]] = out
        
        return derivatives, u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    # Must ensure that the implementation of neural_net_scale is consistent
    # and hopefully correct
    # also, you might not need this function in some datasets
    def neural_net_scale(self, inp): 
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [7]:
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 1, bias=True),
                            )
complex_model = torch.nn.Sequential(
                                    torch.nn.Linear(inp_dimension, 200),
                                    RealToCplx(),
                                    complex_model
                                    )

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:597: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


In [8]:
# Pretrained model
semisup_model_state_dict = cpu_load("./saved_path_inverse_nls/NLS_complex_model_500labeledsamples_jointtrainwith500unlabeledsamples.pth")
parameters = OrderedDict()

# Filter only the parts that I care about renaming (to be similar to what defined in TorchMLP).
inner_part = "network.model."
for p in semisup_model_state_dict:
    if inner_part in p:
        parameters[p.replace(inner_part, "")] = semisup_model_state_dict[p]
complex_model.load_state_dict(parameters)

# pinn = RobustComplexPINN(model=complex_model, loss_fn=mod, index2features=feature_names, scale=False, lb=lb, ub=ub)

<All keys matched successfully>

In [9]:
def closure():
    global X_train, X_train_S, h_train, h_train_S, x_fft, x_PSD, t_fft, t_PSD
    if torch.is_grad_enabled():
        optimizer2.zero_grad(set_to_none=True)
    losses = pinn.loss(X_train, h_train, update_network_params=True, update_pde_params=True)
    l = sum(losses)
    if l.requires_grad: 
        l.backward(retain_graph=True)
    return l

def mtl_closure():
    global X_train, X_train_S, h_train, h_train_S, x_fft, x_PSD, t_fft, t_PSD
    n_obj = 2 # There are two tasks
    losses = pinn.loss(X_train, h_train, update_network_params=True, update_pde_params=True)
    updated_grads = []
    
    for i in range(n_obj):
        optimizer1.zero_grad(set_to_none=True)
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(pinn.parameters()):
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    return sum(losses)

#### Finding the best thresold wrt to the first-epoch loss

In [10]:
pinn = ComplexPINN(model=complex_model, loss_fn=mod, index2features=feature_names, scale=False, lb=lb, ub=ub)

In [11]:
epochs1, epochs2 = 200, 30

# optimizer1 = torch.optim.LBFGS(list(pinn.inp_rpca.parameters())+list(pinn.out_rpca.parameters())+list(pinn.model.parameters())+list(pinn.callable_loss_fn.parameters()), lr=1e-3, line_search_fn='strong_wolfe') also work!
optimizer1 = MADGRAD(pinn.parameters(), lr=5e-7, momentum=0.95)

pinn.train(); best_train_loss = 1e6
print('1st Phase optimization using Adam with PCGrad gradient modification')
for i in range(epochs1):
    optimizer1.step(mtl_closure)
    if (i % 10) == 0 or i == epochs1-1:
        l = mtl_closure()
        print("Epoch {}: ".format(i), l.item())

1st Phase optimization using Adam with PCGrad gradient modification
Epoch 0:  0.02876652032136917
Epoch 10:  0.01757701113820076
Epoch 20:  0.013971677981317043
Epoch 30:  0.012121869251132011
Epoch 40:  0.010788709856569767
Epoch 50:  0.009743858128786087
Epoch 60:  0.00919412449002266
Epoch 70:  0.008641034364700317
Epoch 80:  0.008198561146855354
Epoch 90:  0.00783234927803278
Epoch 100:  0.007600048091262579
Epoch 110:  0.007373086642473936
Epoch 120:  0.007169131189584732
Epoch 130:  0.007018537260591984
Epoch 140:  0.006865114904940128
Epoch 150:  0.006750781089067459
Epoch 160:  0.006645225919783115
Epoch 170:  0.00654009310528636
Epoch 180:  0.00651969388127327
Epoch 190:  0.006536360364407301
Epoch 199:  0.00643113162368536


In [12]:
optimizer2 = torch.optim.LBFGS(pinn.parameters(), lr=1e-1, max_iter=500, 
                               max_eval=int(500*1.25), history_size=150, 
                               line_search_fn='strong_wolfe')

print('2nd Phase optimization using LBFGS')
for i in range(epochs2):
    optimizer2.step(closure)
    if (i % 1) == 0 or i == epochs2-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())
        print(pinn.callable_loss_fn.complex_coeffs().detach().numpy().ravel())

2nd Phase optimization using LBFGS
Epoch 0:  0.0036612548865377903
[ 1.3383147e-05+1.0033953j -1.2189113e-03+0.4990163j]
Epoch 1:  0.0034946873784065247
[-0.00221974+1.001624j   -0.0018779 +0.49773464j]
Epoch 2:  0.0034387956839054823
[-0.00300295+1.0007726j  -0.00191804+0.49700785j]
Epoch 3:  0.003408566117286682
[-0.00347568+1.0013471j  -0.00140845+0.49669296j]
Epoch 4:  0.0033755991607904434
[-0.00316265+1.0005379j  -0.00043661+0.49611694j]
Epoch 5:  0.0033424352295696735
[-0.00301725+1.0001249j   0.00097409+0.49504527j]
Epoch 6:  0.0032808412797749043
[-0.00304911+0.99774384j  0.00243741+0.49505j   ]
Epoch 7:  0.0031319058034569025
[-0.0044014 +0.9966324j   0.00618206+0.49404308j]
Epoch 8:  0.002972292248159647
[-0.00067986+0.9963625j  0.01213049+0.493819j ]
Epoch 9:  0.00274738110601902
[-0.00316572+0.9991702j  0.00972581+0.4951677j]
Epoch 10:  0.0025557056069374084
[-0.00388305+1.0102781j   0.01063359+0.49910393j]
Epoch 11:  0.002373293973505497
[-0.01336616+1.012565j  0.01486195

In [20]:
est_coeffs = pinn.callable_loss_fn.complex_coeffs().detach().numpy().ravel()
print(est_coeffs)
# est_coeffs = npar([8.1524391e-05+0.9902236j, 5.3029885e-03+0.49562332j])
grounds = np.array([1j, 0+0.5j])

errs = []
for i in range(len(grounds)):
    err = est_coeffs[i]-grounds[i]
    errs.append(100*abs(err.imag)/abs(grounds[i].imag))
    
errs = np.array(errs)
errs.mean(), errs.std()

[-0.05378481+1.0157889j   0.0151101 +0.48273963j]


(2.5154829025268555, 0.936591625213623)

In [14]:
# w/o DFT
# clean all
# [0.00019074+0.99958783j 0.00025855+0.5001493j]
# (0.03553926944732666, 0.005677342414855957)

# clean (x, t) | noisy lables
# [0.00063039+1.0004296j  0.00092531+0.50109816j]
# (0.13129711151123047, 0.0883340835571289)

# noisy (x, t) | noisy lables
# 
# 

In [15]:
# U + Noise
# array([0.00118875+0.9984036j, 0.00168767+0.4998621j], dtype=complex64)
# (0.09360909461975098, 0.06603002548217773)
# (Parameter containing:
#  tensor([-0.0012], requires_grad=True),
#  Parameter containing:
#  tensor([-0.0022], requires_grad=True))

# U + Noise | (x, t) + Noise
# (1)
# array([8.1524391e-05+0.9902236j, 5.3029885e-03+0.49562332j], dtype=complex64)
# (0.9264880000000031, 0.051151999999998754)
# (Parameter containing:
#  tensor([0.5776], requires_grad=True),
#  Parameter containing:
#  tensor([-1.0001], requires_grad=True))
# (2) (func=lambda x:(torch.exp(-F.relu(x))))
# [0.00497404+0.99919987j 0.00771778+0.49703848j]
# (0.33615827560424805, 0.25614500045776367)
# (Parameter containing:
#  tensor([-0.1508], requires_grad=True),
#  Parameter containing:
#  tensor([-0.0031], requires_grad=True))

# U + Noise | (x, t) + Noise | "./saved_path_inverse_nls/noisy2_final_finetuned_doublebetarpca_fftthcpinnV2.pth"
# U + Noise | (x, t) + Noise
# (1)
# [0.00297839+1.0081341j  0.00733745+0.50027394j]
# (0.4341006278991699, 0.37931203842163086)
# (Parameter containing:
#  tensor([-0.0341], requires_grad=True),
#  Parameter containing:
#  tensor([-0.0313], requires_grad=True))
# (2) (func=lambda x:(torch.exp(-F.relu(x))))
# [ 0.00067096+1.0004205j -0.00044082+0.5008436j]
# (0.10538101196289062, 0.06333589553833008)
# (Parameter containing:
#  tensor([-0.0026], requires_grad=True),
#  Parameter containing:
#  tensor([0.0011], requires_grad=True))

#### Results

In [16]:
# Noisy Exact & Clean (x, t)
# (0.05885958671569824, 0.021964311599731445)
# array([-0.00046226+0.99919176j, -0.00056662+0.49981552j], dtype=complex64)
# Noisy Exact & Noisy (x, t)
# (0.6996273994445801, 0.01595020294189453)
# array([0.00149273+0.9928442j, 0.00079829+0.5034184j], dtype=complex64)

# Noisy Exact & Clean (x, t) & X_star = X_star-X_star_S
# (0.7112264633178711, 0.00553131103515625)
# array([ 3.449592e-03+1.007057j , -7.125967e-05+0.5035838j], dtype=complex64)
# Noisy Exact & Noisy (x, t) & X_star = X_star-X_star_S
# (0.7093071937561035, 0.0036716461181640625)
# array([ 3.4442921e-03+1.0070564j, -5.4004795e-05+0.5035649j], dtype=complex64)

# Noisy Exact & Clean (x, t) & X_star = X_star_L+1*X_star_S
# (0.1215517520904541, 0.08192658424377441)
# array([-8.2360100e-05+0.99960375j, -6.1671366e-05+0.5010174j], dtype=complex64)
# Noisy Exact & Noisy (x, t) & X_star = X_star_L+1*X_star_S
# (0.511014461517334, 0.25589466094970703)
# array([-0.01472272+1.0076691j, -0.02164156+0.5012756j], dtype=complex64)

# Noisy Exact & Noisy (x, t) & X_train = X_train_L+1*1*X_train_S+beta*NN(X_train_S)
# (0.5050361156463623, 0.1848280429840088)
# array([ 0.00107117+1.0032021j, -0.01103256+0.5034493j], dtype=complex64)
# beta = 0.005178438033908606

# Notes
# X_star = X_star-X_star_S -> Seems robust but not stable
# X_star = X_star_L+X_star_S -> The best?

In [17]:
### New results on Double Beta-RobustFFT ###
# Noisy Exact & Clean (x, t)
# array([-4.01791149e-05+0.9997733j, 1.09734545e-04+0.5006671j], dtype=complex64)
# (0.07804334163665771, 0.05537569522857666)
# (pinn.inp_rpca.beta Parameter containing:
#  tensor([0.0085], requires_grad=True),
#  pinn.out_rpca.beta Parameter containing:
#  tensor([0.0027], requires_grad=True))

# Noisy Exact & Noisy (x, t)
# array([0.00171628+1.0023999j, 0.00308448+0.5002444j], dtype=complex64)
# (0.14443397521972656, 0.09555816650390625)
# (Parameter containing:
#  tensor([0.0029], requires_grad=True),
#  Parameter containing:
#  tensor([0.0012], requires_grad=True))
# --- V2 ---
# array([0.00039933+1.0002806j, 0.00156634+0.5011481j], dtype=complex64)
# (0.12884140014648438, 0.10077953338623047)
# (Parameter containing:
#  tensor([0.9966], requires_grad=True),
#  Parameter containing:
#  tensor([0.9989], requires_grad=True))

### Results on Double Beta-RobustPCA ###
# Noisy Exact & Clean (x, t)
# array([0.00077563+1.0028679j, 0.00166233+0.50137794j], dtype=complex64)
# (0.2811908721923828, 0.005602836608886719)
# (pinn.inp_rpca.beta Parameter containing:
#  tensor([-0.0002], requires_grad=True),
#  pinn.out_rpca.beta Parameter containing:
#  tensor([0.0002], requires_grad=True))

# Noisy Exact & Noisy (x, t)
# array([-0.00045199+1.0037338j, 0.00022461+0.5013247j], dtype=complex64)
# (0.31915903091430664, 0.05421638488769531)
# (pinn.inp_rpca.beta Parameter containing:
#  tensor([-0.0011], requires_grad=True),
#  pinn.out_rpca.beta Parameter containing:
#  tensor([-0.0002], requires_grad=True))

In [18]:
# Expedia hotel recommendation -> Case study data leakage

In [19]:
for i in range(epochs2):
    optimizer2.step(closure)
    if (i % 1) == 0 or i == epochs2-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())
        print(pinn.callable_loss_fn.complex_coeffs().detach().numpy().ravel())

Epoch 0:  0.0008456204668618739
[-0.03843471+1.0095063j   0.02373311+0.48180342j]
Epoch 1:  0.0008089127950370312
[-0.04077225+1.0160183j   0.025232  +0.48101968j]
Epoch 2:  0.0007668425678275526
[-0.04660411+1.019314j    0.02780176+0.47812477j]
Epoch 3:  0.0007366547361016273
[-0.04963372+1.0211407j   0.02907412+0.47664908j]
Epoch 4:  0.0007135001942515373
[-0.0509875 +1.0194252j   0.02618832+0.47897285j]
Epoch 5:  0.0006880747387185693
[-0.05309521+1.0174097j  0.02392182+0.4795014j]
Epoch 6:  0.0006729239830747247
[-0.05398299+1.0159353j   0.02261916+0.47828493j]
Epoch 7:  0.0006715587805956602
[-0.05401897+1.0158172j   0.02255035+0.47826442j]
Epoch 8:  0.0006715587223879993
[-0.05401897+1.0158172j   0.02255035+0.47826442j]
Epoch 9:  0.0006619006744585931
[-0.05380561+1.0148705j   0.02199353+0.47874272j]
Epoch 10:  0.0006531650433316827
[-0.05360104+1.0149411j   0.02056175+0.47954082j]
Epoch 11:  0.0006435803952626884
[-0.05317996+1.0155511j   0.01835804+0.48048675j]
Epoch 12:  0.000